In [1]:
from recordclass import dataobject, make_dataclass, enable_gc, litelist
from random import randint

In [2]:
import tracemalloc
class Tracer:
    def __enter__(self):
        if tracemalloc.is_tracing():
            raise ValueError('nesting tracemalloc is not allowed')
        self.allocated = None
        tracemalloc.start()
        return self
    def __exit__(self, exc_type, exc_value, exc_traceback):
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.allocated = current

In [3]:
N = 24
print(2**N/1000000, "млн.")

16.777216 млн.


### Binary Tree

In [4]:
class Tree(dataobject):
   __fields__ = 'root',

class Node(dataobject): 
    __fields__ = 'left', 'right'

class Leaf(dataobject):
    __fields__ = 'value',

In [5]:
def add_nodes(depth):
    if depth == 0:
        return Leaf(randint(0,10))
    return Node(add_nodes(depth-1), add_nodes(depth-1))

In [6]:
@enable_gc
class TreeGC(dataobject):
   __fields__ = 'root',

@enable_gc
class NodeGC(dataobject): 
    __fields__ = 'left', 'right'

@enable_gc
class LeafGC(dataobject):
    __fields__ = 'value',

In [7]:
def add_nodes_gc(depth):
    if depth == 0:
        return LeafGC(randint(0,10))
    return NodeGC(add_nodes_gc(depth-1), add_nodes_gc(depth-1))

In [8]:
class TreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class NodeSlots:
    __slots__ = 'left', 'right'
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
class LeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [9]:
def add_nodes_slots(depth):
    if depth == 0:
        return LeafSlots(randint(0,10))
    return NodeSlots(add_nodes_slots(depth-1), add_nodes_slots(depth-1))

In [10]:
with Tracer() as t1:
   tree = Tree(add_nodes(N))
print("dataobject:", t1.allocated // 1_000_000, 'Mb')
del tree, t1

dataobject: 939 Mb


In [11]:
%time tree = Tree(add_nodes(N))
%time del tree

CPU times: user 22.1 s, sys: 288 ms, total: 22.4 s
Wall time: 22.4 s
CPU times: user 487 ms, sys: 68 ms, total: 555 ms
Wall time: 554 ms


In [12]:
with Tracer() as t2:
   tree_slots = TreeSlots(add_nodes_slots(N))
print("__slots__:", t2.allocated // 1_000_000, 'Mb')
del tree_slots, t2

__slots__: 2013 Mb


In [13]:
%time tree_slots = TreeSlots(add_nodes_slots(N))
%time del tree_slots

CPU times: user 1min 2s, sys: 560 ms, total: 1min 3s
Wall time: 1min 2s
CPU times: user 780 ms, sys: 148 ms, total: 928 ms
Wall time: 924 ms


In [14]:
with Tracer() as t3:
   tree_gc = TreeGC(add_nodes_gc(N))
print("dataobject+gc:", t3.allocated // 1000000, 'Mb')
del tree_gc, t3

dataobject+gc: 2013 Mb


In [15]:
%time tree_gc = TreeGC(add_nodes_gc(N))
%time del tree_gc

CPU times: user 56.7 s, sys: 672 ms, total: 57.4 s
Wall time: 57.3 s
CPU times: user 581 ms, sys: 116 ms, total: 697 ms
Wall time: 695 ms


In [16]:
import sys
print('Size per instance:')
print('   node:', sys.getsizeof(Node(None,None)),
                  sys.getsizeof(NodeSlots(None,None)), 
                  sys.getsizeof(NodeGC(None,None)))
print('   leaf:', sys.getsizeof(Leaf(None)),
                  sys.getsizeof(LeafSlots(None)),
                  sys.getsizeof(LeafGC(None)))

Size per instance:
   node: 32 64 64
   leaf: 24 56 56


### General Tree

In [17]:
class GTree(dataobject):
   __fields__ = 'root',

class GNode(dataobject): 
    __fields__ = 'children',

class GLeaf(dataobject):
    __fields__ = 'value',

In [18]:
def add_gnodes(depth):
    if depth == 0:
        return GLeaf(randint(0,10))
    children = litelist([])
    gnode = GNode(children)
    children.append(add_gnodes(depth-1))
    children.append(add_gnodes(depth-1))
    children.trim()
    return gnode

In [19]:
with Tracer() as gt1:
   gtree = GTree(add_gnodes(20))
print("dataobject:", gt1.allocated // 1000000, 'Mb')
del gtree, gt1

dataobject: 117 Mb


In [20]:
%time gtree = GTree(add_gnodes(20))
del gtree

CPU times: user 1.69 s, sys: 48.1 ms, total: 1.74 s
Wall time: 1.74 s


In [21]:
class GTreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class GNodeSlots:
    __slots__ = 'children', 
    def __init__(self, children):
        self.children = children
    
class GLeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [22]:
def add_gnodes_slots(depth):
    if depth == 0:
        return GLeafSlots(randint(0,10))
    children = []
    gnode = GNodeSlots(children)
    children.append(add_gnodes_slots(depth-1))
    children.append(add_gnodes_slots(depth-1))
    return gnode

In [23]:
with Tracer() as gt2:
   gtree_slots = GTreeSlots(add_gnodes_slots(20))
print("__slots__:", gt2.allocated // 1000000, 'Mb')
del gtree_slots, gt2

__slots__: 226 Mb


In [24]:
%time gtree_slots = GTreeSlots(add_gnodes_slots(20))
del gtree_slots

CPU times: user 5.55 s, sys: 52 ms, total: 5.6 s
Wall time: 5.6 s
